In [2]:
import polars as pl

In [6]:
gene_lookup = "/Users/nickminor/Documents/dholk_experiments/29418/qiaseq_direct_boosted_with_genes.bed"

In [7]:
gene_df = pl.read_csv(
    gene_lookup, separator="\t", has_header=False
).select(
    ["column_4", "column_7"]
).with_columns(
    pl.col("column_4").str.replace("_RIGHT", "").str.replace("_LEFT", "").alias("column_4")
).unique(
    subset="column_4", keep="first", maintain_order=True
)

In [10]:
long_df = pl.read_ipc("/Users/nickminor/Documents/dholk_experiments/29419/debug.arrow").select(
    ["REGION", "POS", "REF", "ALT", "REF_AA", "ALT_CODON", "ALT_AA", "Amplicon", "Sample ID", "Amplicon-Sample-Contig"]
).with_columns(
    pl.concat_str(
        [pl.col("REF"), pl.col("POS"), pl.col("ALT")], separator="-"
    ).alias("NUC_SUB")
).drop(
    ["REF", "POS", "ALT"]
).join(
    gene_df, how="left", left_on="Amplicon", right_on="column_4"
).rename(
    {"column_7": "Gene"}
).with_columns(
    pl.concat_str(
        [pl.col("REF_AA"), pl.col("ALT_CODON"), pl.col("ALT_AA")], separator="-"
    ).alias("AA_SUB")
).with_columns(
    pl.concat_str(
        [pl.col("Gene"), pl.col("AA_SUB")], separator=": "
    ).alias("AA_SUB")
).with_columns(
    (
        pl.col("ALT_AA").is_null() & pl.col("REF_AA").is_null()
    ).alias("Noncoding")
).with_columns(
    (
        (pl.col("ALT_AA") == pl.col("REF_AA")) &
        (pl.col("Noncoding") == False)
    ).alias("Synonymous")
).with_columns(
    (
        (pl.col("ALT_AA") != pl.col("REF_AA")) &
        (pl.col("Noncoding") == False)
    ).alias("Nonsynonymous")
).drop(
    ["REF_AA", "ALT_AA", "ALT_CODON"]
).rechunk()

Could not mmap compressed IPC file, defaulting to normal read. Toggle off 'memory_map' to silence this warning.


In [11]:
short_df = long_df.unique(
    subset="Amplicon-Sample-Contig", maintain_order=True
).select(
    ["Amplicon-Sample-Contig", "Gene"]
).join(
    
    long_df.select(
        ["Amplicon-Sample-Contig", "NUC_SUB"]
    ).group_by(
        "Amplicon-Sample-Contig", maintain_order=True
    ).agg(
        pl.col("NUC_SUB")
    ).with_columns(
        pl.col("NUC_SUB").list.join(", ").alias("Nucleotide Substitutions")
    ).drop(
        "NUC_SUB"
    ),
    on="Amplicon-Sample-Contig", how="left"
    
).join(
    
    long_df.select(
        ["Amplicon-Sample-Contig", "AA_SUB", "Synonymous"]
    ).filter(
        pl.col("Synonymous")
    ).group_by(
        "Amplicon-Sample-Contig", maintain_order=True
    ).agg(
        pl.col("AA_SUB")
    ).with_columns(
        pl.col("AA_SUB").list.join(", ").alias("Synonymous Mutations")
    ).drop(
        "AA_SUB"
    ),
    on="Amplicon-Sample-Contig", how="left"
    
).join(
    
    long_df.select(
        ["Amplicon-Sample-Contig", "AA_SUB", "Nonsynonymous"]
    ).filter(
        pl.col("Nonsynonymous")
    ).group_by(
        "Amplicon-Sample-Contig", maintain_order=True
    ).agg(
        pl.col("AA_SUB")
    ).with_columns(
        pl.col("AA_SUB").list.join(", ").alias("Nonsynonymous Mutations")
    ).drop(
        "AA_SUB"
    ),
    on="Amplicon-Sample-Contig", how="left"
    
).rechunk()

In [12]:
short_df

Amplicon-Sample-Contig,Gene,Nucleotide Substitutions,Synonymous Mutations,Nonsynonymous Mutations
str,str,str,str,str
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-C, T-177-…",null,null
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-C, T-78-A…",null,null
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-C, C-100-…",null,null
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-C, T-54-C…",null,null
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-C, C-100-…",null,null
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-C, T-78-A…",null,null
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-C, T-78-A…",null,null
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-C, C-100-…",null,null
"""QIAseq_221-SRR…","""ORF1ab""","""A-27-T, C-100-…",null,null
